In [1]:
import datetime as datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('default') # Make the graphs a bit prettier
plt.rcParams['figure.figsize'] = (15, 5)

In [2]:
dsSell2013 = pd.read_csv("sell2013.csv", low_memory=False)
dsSell2014 = pd.read_csv("sell2014.csv", low_memory=False)
dsSell2015 = pd.read_csv("sell2015.csv", low_memory=False)
dsSell2016 = pd.read_csv("sell2016.csv", low_memory=False)
dsSell2017 = pd.read_csv("sell2017.csv", low_memory=False)

In [3]:
#unimos todos los csv y filtramos para que solo queda GBA y CABA
dsSell = dsSell2013.append(dsSell2014).append(dsSell2015).append(dsSell2016).append(dsSell2017)

In [4]:
dsSell.head()

,country_name,created_on,currency,description,expenses,extra,floor,geonames_id,id,image_thumbnail,...,price_per_m2,price_usd_per_m2,properati_url,property_type,rooms,state_name,surface_covered_in_m2,surface_in_m2,surface_total_in_m2,title
0,NaN,2012-08-08,NaN,NaN,850 aprox,NaN,10.0,3436077.0,NaN,http://cf-thumbs.properati.com.ar/Cry7pVKa41t_...,...,NaN,NaN,http://www.properati.com.ar/ag_venta_departame...,apartment,3.0,NaN,NaN,102.0,NaN,NaN
1,NaN,2012-08-08,NaN,NaN,NaN,NaN,11.0,3427467.0,NaN,http://cf-thumbs.properati.com.ar/MwrzW_dkQbqR...,...,NaN,NaN,http://www.properati.com.ar/ah_venta_departame...,apartment,3.0,NaN,NaN,73.0,NaN,NaN
2,NaN,2012-08-08,NaN,NaN,NaN,NaN,NaN,3427467.0,NaN,http://cf-thumbs.properati.com.ar/2wMqxAsh1N2t...,...,NaN,NaN,http://www.properati.com.ar/aj_venta_casa_vill...,house,4.0,NaN,NaN,485.0,NaN,NaN
3,NaN,2012-08-08,USD,NaN,NaN,NaN,NaN,3427467.0,NaN,http://cf-thumbs.properati.com.ar/uMwzJsvexa1C...,...,NaN,315.186246,http://www.properati.com.ar/ak_venta_casa_vill...,house,4.0,NaN,NaN,349.0,NaN,NaN
4,NaN,2012-08-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://cf-thumbs.properati.com.ar/4kPnofvgR6hs...,...,NaN,NaN,http://www.properati.com.ar/ao_venta_departame...,apartment,3.0,NaN,NaN,NaN,NaN,NaN


In [5]:
#Queremos quedarnos con la zona, el municipio y los que esten en CABA y GBA

dsSell['municipio'] = dsSell['place_with_parent_names'].apply(lambda x: x.split('|')[3])
dsSell['zona'] = dsSell['place_with_parent_names'].apply(lambda x: x.split('|')[2])

dsSell['filtro'] = dsSell['zona'].apply(lambda x: ((x=='Capital Federal')or(x=='Bs.As. G.B.A. Zona Norte')or(x=='Bs.As. G.B.A. Zona Sur')or(x=='Bs.As. G.B.A. Zona Oeste')))
dsSell = dsSell[dsSell.filtro]
dsSell.drop('filtro', axis=1, inplace=True)

In [6]:
#Como nos importa mucho el precio con respecto a otras cosas vamos a sacar a los que tienen esa columna en NaN

dsSell['tiene_precio'] = dsSell['price'].apply(lambda x: pd.notnull(x))
dsSell = dsSell[dsSell.tiene_precio]
dsSell.drop('tiene_precio', axis=1, inplace=True)

In [7]:
#convertimos el formato de la fecha
dsSell['year'] =pd.to_numeric(dsSell['created_on'].map( lambda x: x.split('-')[0]))
dsSell['month'] = pd.to_numeric(dsSell['created_on'].map( lambda x: x.split('-')[1]))
dsSell['created_on'] = dsSell['created_on'] .map(lambda x: x.split('-')[0]+x.split('-')[1]+x.split('-')[2])
dsSell

,country_name,created_on,currency,description,expenses,extra,floor,geonames_id,id,image_thumbnail,...,rooms,state_name,surface_covered_in_m2,surface_in_m2,surface_total_in_m2,title,municipio,zona,year,month
3,NaN,20120808,USD,NaN,NaN,NaN,NaN,3427467.0,NaN,http://cf-thumbs.properati.com.ar/uMwzJsvexa1C...,...,4.0,NaN,NaN,349.0,NaN,NaN,General San Martín,Bs.As. G.B.A. Zona Norte,2012,8
6,NaN,20120808,USD,NaN,NaN,NaN,NaN,NaN,NaN,http://cf-thumbs.properati.com.ar/BJum1WvpYb6P...,...,NaN,NaN,NaN,153.0,NaN,NaN,General San Martín,Bs.As. G.B.A. Zona Norte,2012,8
10,NaN,20120808,USD,NaN,NaN,NaN,NaN,NaN,NaN,http://cf-thumbs.properati.com.ar/77Q-mNgsgx7O...,...,NaN,NaN,NaN,85.0,NaN,NaN,Tigre,Bs.As. G.B.A. Zona Norte,2012,8
11,NaN,20120808,USD,NaN,NaN,NaN,NaN,NaN,NaN,http://cf-thumbs.properati.com.ar/DuUbXCV9Xzos...,...,4.0,NaN,NaN,383.0,NaN,NaN,Pilar,Bs.As. G.B.A. Zona Norte,2012,8
12,NaN,20120808,USD,NaN,NaN,NaN,NaN,NaN,NaN,http://cf-thumbs.properati.com.ar/UAN_2iHtE0hA...,...,NaN,NaN,NaN,255.0,NaN,NaN,Tigre,Bs.As. G.B.A. Zona Norte,2012,8
13,NaN,20120808,USD,NaN,NaN,NaN,5.0,NaN,NaN,http://cf-thumbs.properati.com.ar/pJ0Vap4MCjhW...,...,4.0,NaN,NaN,240.0,NaN,NaN,Las Cañitas,Capital Federal,2012,8
14,NaN,20120809,USD,NaN,NaN,NaN,NaN,3433918.0,NaN,http://cf-thumbs.properati.com.ar/7LLinQKdItkV...,...,5.0,NaN,NaN,250.0,NaN,NaN,Flores,Capital Federal,2012,8
16,NaN,20120815,USD,NaN,NaN,NaN,NaN,3433910.0,NaN,http://cf-thumbs.properati.com.ar/folXLAuUibJC...,...,1.0,NaN,NaN,25.0,NaN,NaN,Floresta,Capital Federal,2012,8
17,NaN,20120815,USD,NaN,NaN,NaN,NaN,NaN,NaN,http://cf-thumbs.properati.com.ar/QIshWpCxB_sU...,...,4.0,NaN,NaN,180.0,NaN,NaN,Parque Centenario,Capital Federal,2012,8
18,NaN,20120817,USD,NaN,NaN,NaN,NaN,3431333.0,NaN,http://cf-thumbs.properati.com.ar/vfIoxXFjrZY7...,...,3.0,NaN,NaN,108.0,NaN,NaN,Liniers,Capital Federal,2012,8


In [8]:
#vemos como evolucionan algunos features en el tiempo para la venta
cantidadSellEnAnios = dsSell.groupby('created_on').count().reset_index().rename(columns={'price_aprox_usd':'cantidad'}).sort_values(by = 'created_on',ascending= True)[['created_on', 'cantidad']]
preciosSellEnAnios = dsSell.groupby('created_on').mean().reset_index().sort_values(by = 'created_on',ascending= True)[['created_on', 'price_aprox_usd']]

In [9]:
#Cantidad de viviendas vendidas en dolares y pesos a lo largo del tiempo
cantidadSellEnAniosEnDolares = dsSell[dsSell['currency'] == 'USD'].groupby('created_on').count().reset_index().rename(columns={'price':'cantidad'}).sort_values(by = 'created_on',ascending= True)[['created_on', 'cantidad']]
cantidadSellEnAniosEnPesos = dsSell[dsSell['currency'] == 'ARS'].groupby('created_on').count().reset_index().rename(columns={'price':'cantidad'}).sort_values(by = 'created_on',ascending= True)[['created_on', 'cantidad']]

In [10]:
#contamos la cantidad de publicaciones para cada año
Sell2012 = dsSell[dsSell.created_on.str.contains('2012')]
Sell2013 = dsSell[dsSell.created_on.str.contains('2013')]
Sell2014 = dsSell[dsSell.created_on.str.contains('2014')]
Sell2015 = dsSell[dsSell.created_on.str.contains('2015')]
Sell2016 = dsSell[dsSell.created_on.str.contains('2016')]
Sell2017 = dsSell[dsSell.created_on.str.contains('2017')]
print (Sell2012.id.count())
print (Sell2013.id.count())
print (Sell2014.id.count())
print (Sell2015.id.count())
print (Sell2016.id.count())
print (Sell2017.id.count())

3
357
3599
8315
59443
76030


In [11]:
#generamos un df con los precios por m2 en usd para cada zona por año
dSell2012 = Sell2012.groupby('zona').mean().reset_index()[['zona','price_usd_per_m2']]
dSell2013 = Sell2013.groupby('zona').mean().reset_index()[['zona','price_usd_per_m2']]
dSell2014 = Sell2014.groupby('zona').mean().reset_index()[['zona','price_usd_per_m2']]
dSell2015 = Sell2015.groupby('zona').mean().reset_index()[['zona','price_usd_per_m2']]
dSell2016 = Sell2016.groupby('zona').mean().reset_index()[['zona','price_usd_per_m2']]
dSell2017 = Sell2017.groupby('zona').mean().reset_index()[['zona','price_usd_per_m2']]
dSell_price_m2_per_year = Sell2012.groupby('zona').mean().reset_index()[['zona','price_usd_per_m2']]
dSell_price_m2_per_year['price_usd_per_m2-2013'] = dSell2013['price_usd_per_m2']
dSell_price_m2_per_year['price_usd_per_m2-2014'] = dSell2014['price_usd_per_m2']
dSell_price_m2_per_year['price_usd_per_m2-2015'] = dSell2015['price_usd_per_m2']
dSell_price_m2_per_year['price_usd_per_m2-2016'] = dSell2016['price_usd_per_m2']
dSell_price_m2_per_year['price_usd_per_m2-2017'] = dSell2017['price_usd_per_m2']
dSell_price_m2_per_year.rename(columns={'price_usd_per_m2':'price_usd_per_m2-2012'},inplace = True)
dSell_price_m2_per_year.head()

,zona,price_usd_per_m2-2012,price_usd_per_m2-2013,price_usd_per_m2-2014,price_usd_per_m2-2015,price_usd_per_m2-2016,price_usd_per_m2-2017
0,Bs.As. G.B.A. Zona Norte,1381.806414,1851.048579,1211.650669,1403.292299,1634.885487,1847.432807
1,Bs.As. G.B.A. Zona Oeste,805.626271,1463.920457,1185.779292,1282.050329,1431.704285,1295.971166
2,Bs.As. G.B.A. Zona Sur,1206.732371,1500.689534,1191.057100,1233.497276,1343.147175,1512.130051
3,Capital Federal,2115.121019,2509.180265,2193.455416,2457.443527,2587.740926,3091.645588


In [12]:
 
cantidadEnVentaPorMes = dsSell.groupby(['month', 'year']).count().reset_index().rename(columns={'price_aprox_usd':'cantidad'})[['year', 'month', 'cantidad']]
precioEnEventaPorMes = dsSell.groupby(['month', 'year']).mean().reset_index()[['year', 'month', 'price_aprox_usd']]

In [13]:
dsSell['cochera'] = dsSell.description.str.contains('cochera')
dsSell['pileta'] = dsSell.description.str.contains('pileta')
dsSell['sum'] = dsSell.description.str.contains('sum')
dsSell['aire acondicionado'] = dsSell.description.str.contains('aire acondicionado')
dsSell['calefaccion'] = dsSell.description.str.contains('calefaccion')
dsSell['seguridad'] = dsSell.description.str.contains('seguridad')
dsSell['estrenar'] = dsSell.description.str.contains('estrenar')
dsSell['emprendimiento'] = dsSell.description.str.contains('emprendimiento')


In [14]:
#convertimos los valores false , true y nan de las busquedas anteriores

dsSell['cochera'] = dsSell['cochera'].map(lambda x: 0 if x == 'False' else 1 if x == 'True' else 0.5)
dsSell['pileta'] = dsSell['pileta'].map(lambda x: 0 if x == 'False' else 1 if x == 'True' else 0.5)
dsSell['sum'] = dsSell['sum'].map(lambda x: 0 if x == 'False' else 1 if x == 'True' else 0.5)
dsSell['aire acondicionado'] = dsSell['aire acondicionado'].map(lambda x: 0 if x == 'False' else 1 if x == 'True' else 0.5)
dsSell['calefaccion'] = dsSell['calefaccion'].map(lambda x: 0 if x == 'False' else 1 if x == 'True' else 0.5)
dsSell['seguridad'] = dsSell['seguridad'].map(lambda x: 0 if x == 'False' else 1 if x == 'True' else 0.5)
dsSell['estrenar'] = dsSell['estrenar'].map(lambda x: 0 if x == 'False' else 1 if x == 'True' else 0.5)
dsSell['emprendimiento'] = dsSell['emprendimiento'].map(lambda x: 0 if x == 'False' else 1 if x == 'True' else 0.5)

In [15]:
#drop de columnas
dsSell.drop('price_per_m2', axis = 1, inplace = True)
dsSell.drop('place_with_parent_names', axis = 1, inplace = True)
dsSell.drop('created_on', axis = 1, inplace = True)
dsSell.drop('description', axis = 1, inplace = True)
dsSell.drop('currency', axis=1, inplace=True)
dsSell.drop('extra', axis=1, inplace=True)
dsSell.drop('id', axis=1, inplace=True)
dsSell.drop('image_thumbnail', axis=1, inplace=True)
dsSell.drop('properati_url', axis=1, inplace=True)
dsSell.drop('title', axis=1, inplace=True)
dsSell.drop('lat-lon', axis=1, inplace=True)
dsSell.drop('operation', axis=1, inplace=True)
dsSell.drop('price', axis=1, inplace=True)
dsSell.drop('price_aprox_local_currency', axis=1, inplace=True)
dsSell.drop('country_name', axis=1, inplace=True)

In [16]:
dsSell.columns

Index(['expenses', 'floor', 'geonames_id', 'lat', 'lon', 'place_name',
       'price_aprox_usd', 'price_usd_per_m2', 'property_type', 'rooms',
       'state_name', 'surface_covered_in_m2', 'surface_in_m2',
       'surface_total_in_m2', 'municipio', 'zona', 'year', 'month', 'cochera',
       'pileta', 'sum', 'aire acondicionado', 'calefaccion', 'seguridad',
       'estrenar', 'emprendimiento'],
      dtype='object')

In [17]:
#preprocessing

#eliminamos todo lo q no es numero de expenses
dsSell['expenses'] = dsSell['expenses'].str.extract('(\d+)')

M:\Users\Krion\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)


In [18]:
#convertimos a float

dsSell['expenses'] =pd.to_numeric(dsSell.expenses, errors='coerce').notnull()


In [19]:
#one hot encoding de los place name. me suena mal muchas columnas. performance

#primera aproximacion teniendo en cuenta solamente la zona, no solo los barrios 
one_hot = pd.get_dummies(dsSell['zona'])
dsSell.drop('zona', axis=1, inplace=True)
dsSell = dsSell.join(one_hot)

one_hot = pd.get_dummies(dsSell['property_type'])
dsSell.drop('property_type', axis=1, inplace=True)
dsSell = dsSell.join(one_hot)

dsSell.drop('place_name', axis=1, inplace=True)

dsSell.drop('municipio', axis=1, inplace=True)
dsSell.drop('state_name', axis=1, inplace=True)

In [20]:
#filleamos los nan con el promedio de las columnas
dsSell = dsSell.apply(lambda x: x.fillna(x.mean()),axis=0)

In [21]:
#normalize data

In [23]:
#creamos un set de test y otro de entrenamiento. el test de entrenamiento sera de un 10% del original 

dsSell['is_true'] = np.random.uniform(0,1,len(dsSell)) <= 0.1
dsSellTrain, dsSellTest = dsSell[dsSell['is_true']==True], dsSell[dsSell['is_true']==False]
del dsSell['is_true']
del dsSellTrain['is_true']
del dsSellTest['is_true']

dsSellTrain_y = dsSellTrain['price_aprox_usd']
dsSellTrain_x = dsSellTrain.drop('price_aprox_usd', axis=1)
dsSellTest_y = dsSellTest['price_aprox_usd']
dsSellTest_x = dsSellTest.drop('price_aprox_usd', axis=1)

In [26]:
#normalize data
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

dsSellTrain_x = preprocessing.normalize(dsSellTrain_x)

scaler = StandardScaler()
scaler.fit(dsSellTrain_x) 
dsSellTrain_x= scaler.transform(dsSellTrain_x)


In [ ]:
model.get_params()

In [27]:
from sklearn.model_selection import RandomizedSearchCV
from numpy import random
from sklearn import linear_model
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import make_scorer

import scipy
import time

print("[INFO] tuning hyperparameters via grid search")

param_dist = {'loss' : ['squared_epsilon_insensitive'],
              'learning_rate' : ['optimal', 'invscaling'] , 'eta0' : [0.01,0.001,0.1], 'power_t' : [0.25,0.60],
                'alpha' : [1,10,0.01,0.0001,1000]} 

model = linear_model.SGDRegressor()
grid = RandomizedSearchCV(model, param_distributions=param_dist, n_iter=10)

start = time.time()
print("[INFO] fitting stage")
grid = grid.fit(dsSellTrain_x , dsSellTrain_y)
acc = grid.best_score_

print("[INFO] grid search took {:.2f} seconds".format(time.time() - start))
print("[INFO] grid search accuracy: {:.2f}%".format(acc * 100))
print("[INFO] grid search best parameters: {}".format(grid.best_params_))

[INFO] tuning hyperparameters via grid search
[INFO] fitting stage
[INFO] grid search took 62.21 seconds
[INFO] grid search accuracy: 8.62%
[INFO] grid search best parameters: {'power_t': 0.25, 'loss': 'squared_epsilon_insensitive', 'learning_rate': 'optimal', 'eta0': 0.01, 'alpha': 1}
